In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo de gráficos con un tema oscuro personalizado
custom_dark_style = {
    'axes.facecolor': '#1f1f1f',    # Color de fondo de los ejes
    'axes.edgecolor': '#ffffff',     # Color del borde de los ejes
    'axes.labelcolor': '#ffffff',    # Color de las etiquetas de los ejes
    'xtick.color': '#ffffff',        # Color de las marcas del eje x
    'ytick.color': '#ffffff',        # Color de las marcas del eje y
    'text.color': '#ffffff',         # Color del texto
    'figure.facecolor': '#121212',   # Color de fondo de la figura
    'grid.color': '#555555',         # Color de las líneas de la cuadrícula
}

plt.style.use(custom_dark_style)

# Ruta del archivo CSV
file_path = 'datos/export_2024_05_22_17_21.csv'

# Leer el archivo CSV
data = pd.read_csv(file_path)

# Convertir las columnas 'start' y 'end' a tipo datetime con manejo de errores
data['start'] = pd.to_datetime(data['start'], errors='coerce')
data['end'] = pd.to_datetime(data['end'], errors='coerce')


data['high'] = pd.to_numeric(data['high'], errors='coerce')
data['low'] = pd.to_numeric(data['low'], errors='coerce')

# eliminar ceros y ausentes:
data[['vol','ask', 'bid', 'open', 'high', 'low', 'close']] = data[['vol','ask', 'bid', 'open', 'high', 'low', 'close']].replace(0, np.nan)
data.dropna(subset=['vol','ask', 'bid', 'open', 'high', 'low', 'close'], inplace=True)
data.dropna(subset=['start', 'end'], inplace=True)

# Calcular el promedio y la desviación estándar de la columna 'vol'
vol_mean = data['vol'].mean()
vol_std = data['vol'].std()

num_std = 3  # Por ejemplo, consideramos como límites 3 desviaciones estándar

# Filtrar los datos para mantener solo aquellos dentro del rango específico de desviaciones estándar del promedio
filtered_data = data[
                     (data['vol'] >= vol_mean - num_std * vol_std) & 
                     (data['vol'] <= vol_mean + num_std * vol_std)]



C:\Users\Javier-pc\AppData\Local\Temp\ipykernel_13732\3303537891.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['start'] = pd.to_datetime(data['start'], errors='coerce')


In [9]:
filtered_data['tendencia'] = np.where(filtered_data['close'] >= filtered_data['open'], 1, -1)

# Crear una nueva columna llamada 'condicion' inicializada con valores nulos
filtered_data['condicion'] = np.nan

# Iterar sobre las filas del filtered_dataFrame
for i in range(1, len(filtered_data)):
    # Verificar que el índice esté dentro del rango del DataFrame
    if i - 1 in filtered_data.index:
        # Verificar la tendencia de la fila anterior
        if filtered_data.loc[i - 1, 'tendencia'] == 1:
            # Verificar si el open actual es mayor al close de la fila anterior
            if filtered_data.loc[i, 'open'] > filtered_data.loc[i - 1, 'close']:
                filtered_data.loc[i, 'condicion'] = 1
            else:
                filtered_data.loc[i, 'condicion'] = 0
        elif filtered_data.loc[i - 1, 'tendencia'] == -1:
            # Verificar si el open actual es menor al close de la fila anterior
            if filtered_data.loc[i, 'open'] < filtered_data.loc[i - 1, 'close']:
                filtered_data.loc[i, 'condicion'] = 1
            else:
                filtered_data.loc[i, 'condicion'] = 0


C:\Users\Javier-pc\AppData\Local\Temp\ipykernel_13732\3835832625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['tendencia'] = np.where(filtered_data['close'] >= filtered_data['open'], 1, -1)
C:\Users\Javier-pc\AppData\Local\Temp\ipykernel_13732\3835832625.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['condicion'] = np.nan


KeyError: 20